In [121]:
import instructor
from anthropic import Anthropic
from pydantic import BaseModel, Field
from dotenv import load_dotenv

anthropic = Anthropic()

load_dotenv()

True

In [ ]:
queries = [
    "how does a hepatic adenoma look like on MRI?",
    "penumbra ischemic stroke perfuson imaging",
    "how to differentiate between a hepatic adenoma and focal nodular hyperplasia on MRI?",
    "how does japanese encephalitis look like on MRI?",
    "what is CLIPPERS?",
    "What is a normal callosal angle, and how do I measure it?",
    "Can you tell me more about the MTA score?",
    "what are the criteria of an liver abscess",
    "pneumonitis",
    "how does the penumbra look in ct perfusion?"
]

In [102]:
class SearchTerm(BaseModel):
    text: str = Field(..., title="The search term extracted from the provided text. The search term should be the underlying disease or condition that the user is asking about.")


client = instructor.from_anthropic(Anthropic())

def get_search_term(query: str) -> SearchTerm:
# note that client.chat.completions.create will also work
    resp = client.messages.create(
        model="claude-3-5-haiku-latest",
        max_tokens=1024,
        messages=[
            {
                "role": "system",
                "content": "Extract a search term from the provided text.",},
            {
                "role": "user",
                "content": f"text: {query}",
            }
        ],
        response_model=SearchTerm,
    )
    
    print("SearchTerm for query: ", query, "is: ", resp.text)

    return resp

In [103]:
import requests
from bs4 import BeautifulSoup

def search_radiopaedia(search_query: str):
    url = 'https://radiopaedia.org/search'


    params = {
        'lang': 'us',
        'q': search_query,
        'scope': 'articles'
    }

    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'accept-language': 'de-DE,de;q=0.7',
        'cache-control': 'max-age=0',
        'cookie': '*radiopaedia*session=FK2kLR8%2Bhkzg8D5gtP4N4x7y7QoL55kFD40yMiPSOY7%2FN2z7QD85jgYgOl36adv2O4KM%2Buk9%2BIxtFUDX3LTHoSSOeoykDZyd%2FLqq5OPox0tzbn1URW4n5oZhowZCpw9MHPRvZ%2FAIDDhAtaJxz7Ue3hBb%2BjcKRloPcYbUjUBOi4KvtINrFnKpfey%2B13AgAz8bcuzLURQiO07IXbnVRackkRXpgG0mBcjh8n1Ap849t33s81SgYXTXFGeBit4JVhTtqLGrUa%2F%2FIXFZZyKREUk9b8kQYmvUIRtE9Fmr1Rd43JQOa60hTs%2F4Vhh%2F1rClz1pAgZrzAIEDbzx%2Bvz1CKDU%2FhHlydPYIcvrhZXOQ6TKhRDV8bfrfKjjToXAH9vyFbq4VxlnAnyoCA4JBFetkzZbrqrZSYsM%2F%2BF7LW8Swh92pLtO%2BwAps555wlyXnQKDbrapf%2FNaABd3%2Bk301t64uwC1n4nkq3Z7rIG409N%2FFfzylkrNs1eOArX1j%2FqkilucHBXMt--A7TmaE8Ut8M5kXCo--txKn9l3AgN8d8znLnEjAKw%3D%3D',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate', 
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
    }

    response = requests.get(url, params=params, headers=headers)

    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [104]:
def structure_search_result(result, i): 
    return {
        "id": i,
        "title": result.find(class_='search-result-title').text.strip(),
        "body": result.find(class_='search-result-body').text.strip(),
        "href": result['href']
    }

In [105]:
def format_search_results(results):
    def format_single_search_result(result): 
        return "ID: " + str(result['id']) + "\n\n" + "Title: " + result['title'] + "\n\n" + "Body: " + result['body'] + "\n"
    
    res_string = "\n========================\n\n".join([format_single_search_result(result) for result in results])
    
    print(res_string)
    
    return res_string

In [106]:
class BestSearchResult(BaseModel):
    chain_of_thought: str = Field(..., title="The reason why this is the best search result for the given query.")
    id: int = Field(..., title="The id of the best matching search result for the given query.")


client = instructor.from_anthropic(Anthropic())

def get_best_result(query: str, search_results) -> SearchTerm:
# note that client.chat.completions.create will also work
    resp = client.messages.create(
        model="claude-3-5-haiku-latest",
        max_tokens=1024,
        messages=[
            {
                "role": "system",
                "content": "return the id of the best matching search result for the given query."},
            {
                "role": "user",
                "content": f"query: {query}\n\nsearch_results: {format_search_results(search_results)}",
            }
        ],
        response_model=BestSearchResult,
    )

    print("BestSearchResult for query: ", query, "is: ", search_results[resp.id])
    
    return resp

In [130]:
def get_article_text(url): 
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'accept-language': 'de-DE,de;q=0.7',
        'cache-control': 'max-age=0',
        'cookie': '*radiopaedia*session=FK2kLR8%2Bhkzg8D5gtP4N4x7y7QoL55kFD40yMiPSOY7%2FN2z7QD85jgYgOl36adv2O4KM%2Buk9%2BIxtFUDX3LTHoSSOeoykDZyd%2FLqq5OPox0tzbn1URW4n5oZhowZCpw9MHPRvZ%2FAIDDhAtaJxz7Ue3hBb%2BjcKRloPcYbUjUBOi4KvtINrFnKpfey%2B13AgAz8bcuzLURQiO07IXbnVRackkRXpgG0mBcjh8n1Ap849t33s81SgYXTXFGeBit4JVhTtqLGrUa%2F%2FIXFZZyKREUk9b8kQYmvUIRtE9Fmr1Rd43JQOa60hTs%2F4Vhh%2F1rClz1pAgZrzAIEDbzx%2Bvz1CKDU%2FhHlydPYIcvrhZXOQ6TKhRDV8bfrfKjjToXAH9vyFbq4VxlnAnyoCA4JBFetkzZbrqrZSYsM%2F%2BF7LW8Swh92pLtO%2BwAps555wlyXnQKDbrapf%2FNaABd3%2Bk301t64uwC1n4nkq3Z7rIG409N%2FFfzylkrNs1eOArX1j%2FqkilucHBXMt--A7TmaE8Ut8M5kXCo--txKn9l3AgN8d8znLnEjAKw%3D%3D',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate', 
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.select("#content > div.body.user-generated-content")[0].text.strip()

In [142]:
from utils import answer_question

In [141]:
query = "What is a normal callosal angle, and how do I measure it?"

answer = answer_question(query)

display(Markdown(answer))

SearchTerm for query:  What is a normal callosal angle, and how do I measure it? is:  callosal angle
ID: 0

Title: Callosal angle

Body: The callosal angle has been proposed as a useful marker of patients with idiopathic normal pressure hydrocephalus (iNPH), helpful in distinguishing these patients from those with ex-vacuo ventriculomegaly (see hydrocephalus versus atrophy). 

It should be noted that there is nothing magical abou...


ID: 1

Title: Hydrocephalus vs atrophy

Body: Distinguishing primary hydrocephalus from atrophy resulting in compensatory enlargement of the CSF spaces as the cause of ventriculomegaly can be, at times, challenging in image interpretation.

Radiographic features

Features that favor hydrocephalus include:

dilatation of the temporal horns

...


ID: 2

Title: iNPH Radscale

Body: The iNPH Radscale is a reproducible semiquantified grading scale for the imaging findings of normal pressure hydrocephalus (NPH).

Scale

The iNPH Radscale uses the following ima

Based on the context:

A normal callosal angle is typically between 100-120°. 

To measure the callosal angle:
1. Use a coronal image
2. The image should be perpendicular to the anterior commissure - posterior commissure (AC-PC) plane
3. The measurement should be taken at the level of the posterior commissure

By contrast, patients with idiopathic normal pressure hydrocephalus (iNPH) tend to have smaller angles, usually between 50-80°.